In [9]:
import pandas as pd
import re
import numpy as np
from scipy import stats
import scipy as sp
from sklearn import metrics
from bayesian_player_ratings import *

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.precision', 4)

Load Match Summary Data

In [10]:
match_summary = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/match_summary.csv")
match_summary = score_col_splitter(match_summary, "Q4_Score")
match_summary['Season'] = match_summary['Match_ID'].apply(lambda x: int(x[:4]))

player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/scored_player_stats_v2.csv")
player_stats = create_additional_player_stats_variables(player_stats)
player_stats['exp_vaep_value'] = player_stats['exp_vaep_value'].fillna(0)
player_stats['exp_offensive_value'] = player_stats['exp_offensive_value'].fillna(0)
player_stats['exp_defensive_value'] = player_stats['exp_defensive_value'].fillna(0)

match_stats = aggregate_player_to_match_stats(player_stats)

match_summary_stats = match_summary.merge(match_stats, how = "left", on = "Match_ID")

In [11]:
def calculate_probabilities(rating_diff, std):
    
    diff = stats.norm(loc = rating_diff, scale = std)
    
    away_prob = diff.cdf(0)
    draw_prob = diff.pdf(0)
    home_prob = 1 - away_prob - draw_prob
    
    return home_prob, draw_prob, away_prob

In [12]:
def get_round_predictions(player_stats, round_id, match_std = 35):
    
    round_match_list = list(player_stats[player_stats['Round_ID'] == round_id]['Match_ID'].unique())
    
    match_projections_dict = {}
    for match_id in round_match_list:
        match_projections_dict[match_id] = {}
        home_team, away_team = get_teams(match_id)
        home_rating = ratings.team_values[round_id][ratings.team_values[round_id]['Team'] == home_team]['Rating'].iloc[0]
        away_rating = ratings.team_values[round_id][ratings.team_values[round_id]['Team'] != home_team]['Rating'].iloc[0]
        rating_difference = home_rating - away_rating
        home_prob, draw_prob, away_prob = calculate_probabilities(rating_difference, match_std)
        match_projections_dict[match_id]['projected_diff'] = rating_difference
        # match_projections_dict[match_id]['projected_prob'] = home_prob
        
        home_exp_vaep = player_stats[(player_stats['Match_ID'] == match_id) & (player_stats['Team'] == player_stats['Home_Team'])]['exp_vaep_value'].sum()
        away_exp_vaep = player_stats[(player_stats['Match_ID'] == match_id) & (player_stats['Team'] != player_stats['Home_Team'])]['exp_vaep_value'].sum()
        exp_vaep_result = home_exp_vaep - away_exp_vaep
        # match_projections_dict[match_id]['exp_vaep_diff'] = exp_vaep_result        
        
        home_score = match_summary_stats[(match_summary_stats['Match_ID'] == match_id)]['Home_Score'].sum()
        away_score = match_summary_stats[(match_summary_stats['Match_ID'] == match_id)]['Away_Score'].sum()
        result = home_score - away_score
        match_projections_dict[match_id]['margin'] = result
        
        match_projections_dict[match_id]['mae'] = abs(match_projections_dict[match_id]['margin'] - match_projections_dict[match_id]['projected_diff'])

    return match_projections_dict


In [13]:
player_stats_2021 = player_stats[player_stats['Season'] == 2021]
match_summary_stats_2021 = match_summary_stats[match_summary_stats['Season'] == 2021]

In [14]:
ratings = TeamRatings(prior_weight = 1.8)
ratings.calculate_home_advantage(match_summary_stats_2021)
ratings.initialise_player_values(player_stats_2021)
ratings.initialise_team_values(player_stats_2021)
ratings.calculate_home_advantage(match_summary_stats_2021)
ratings.latest_round

'Start'

Rounds

In [15]:
round_projections_dict = {}
round_mae_dict = {}

In [16]:
round_id = '202101'
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 20.57
Overall MAE: 20.57


,Team,Rating,Offensive_Value,Defensive_Value
13,Richmond,3.7541,11.9349,8.1808
10,Melbourne,2.8527,9.0964,6.2437
16,West Coast,2.7887,10.4664,7.6777
12,Port Adelaide,2.7859,11.7402,8.9543
17,Western Bulldogs,2.6695,9.5806,6.9112
15,Sydney,2.0804,9.7485,7.6682
8,Greater Western Sydney,1.1833,10.4971,9.3138
4,Essendon,0.2944,9.1347,8.8403
0,Adelaide,0.0000,0.0000,0.0000
6,Geelong,0.0000,0.0000,0.0000


In [17]:
round_id = "202102"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)


Round MAE: 25.14
Overall MAE: 22.85


,Team,Rating,Offensive_Value,Defensive_Value
13,Richmond,6.6755,20.7919,14.1164
10,Melbourne,6.1548,19.6530,13.4982
12,Port Adelaide,4.6833,19.3440,14.6606
7,Gold Coast,4.5917,17.6017,13.0100
17,Western Bulldogs,3.6506,19.4382,15.7876
15,Sydney,1.3141,18.5548,17.2407
16,West Coast,1.2618,18.9715,17.7097
5,Fremantle,1.2128,18.2377,17.0248
0,Adelaide,0.5582,11.4547,10.8965
6,Geelong,0.1202,8.4762,8.3560


In [18]:
round_id = "202103"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)


Round MAE: 39.35
Overall MAE: 28.35


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,13.2773,31.1760,17.8987
10,Melbourne,8.8489,28.1256,19.2768
15,Sydney,4.3175,26.5930,22.2755
7,Gold Coast,3.3199,23.8101,20.4902
16,West Coast,3.2443,26.0734,22.8291
13,Richmond,2.8732,25.0184,22.1453
12,Port Adelaide,2.1063,25.5840,23.4777
0,Adelaide,1.3150,19.3356,18.0206
2,Carlton,1.2884,26.2276,24.9392
4,Essendon,1.1531,20.9828,19.8296


In [19]:
round_id = "202104"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)


Round MAE: 13.39
Overall MAE: 24.61


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,15.2141,38.8076,23.5935
10,Melbourne,10.4368,34.5264,24.0895
15,Sydney,5.5517,33.4331,27.8814
2,Carlton,5.5047,33.4666,27.9619
12,Port Adelaide,4.5142,32.7261,28.2119
0,Adelaide,3.5150,27.6744,24.1594
5,Fremantle,0.0665,28.2352,28.1687
13,Richmond,-0.0327,26.4724,26.5051
4,Essendon,-0.6281,27.4937,28.1218
16,West Coast,-0.7247,27.5855,28.3102


In [20]:
round_id = "202105"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 35.67
Overall MAE: 26.82


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,18.7335,43.2553,24.5218
10,Melbourne,15.0434,37.9149,22.8716
13,Richmond,6.6351,37.4529,30.8178
2,Carlton,4.9135,37.6163,32.7028
12,Port Adelaide,4.1034,34.4185,30.3151
15,Sydney,3.7708,35.9096,32.1387
6,Geelong,2.3376,27.7849,25.4474
0,Adelaide,1.6737,33.8877,32.2140
1,Brisbane Lions,1.6699,35.6723,34.0023
5,Fremantle,1.5497,35.6557,34.1061


In [21]:
round_id = "202106"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 36.79
Overall MAE: 28.49


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,21.0018,51.3912,30.3894
10,Melbourne,14.1135,43.4721,29.3586
6,Geelong,7.8026,36.0280,28.2253
12,Port Adelaide,6.4242,41.4842,35.0601
5,Fremantle,6.2749,41.1852,34.9104
13,Richmond,5.3971,40.8808,35.4837
2,Carlton,3.8279,43.6479,39.8200
1,Brisbane Lions,2.0973,43.2871,41.1898
0,Adelaide,0.8393,39.5417,38.7023
7,Gold Coast,-0.0031,39.0699,39.0730


In [22]:
round_id = "202107"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 32.08
Overall MAE: 29.00


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,16.7103,50.4999,33.7895
10,Melbourne,14.6536,47.8309,33.1774
6,Geelong,10.4452,39.8359,29.3908
13,Richmond,7.0487,44.6353,37.5866
1,Brisbane Lions,5.2707,44.1793,38.9086
2,Carlton,4.9979,50.2361,45.2382
12,Port Adelaide,2.3986,39.4089,37.0103
5,Fremantle,1.0616,40.0062,38.9446
7,Gold Coast,0.0171,43.5682,43.5511
16,West Coast,-1.5114,36.8154,38.3268


In [23]:
round_id = "202108"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 21.39
Overall MAE: 28.05


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,17.0397,54.6324,37.5927
6,Geelong,14.6833,43.6321,28.9488
10,Melbourne,12.9839,49.9455,36.9616
12,Port Adelaide,7.4609,50.1058,42.6448
1,Brisbane Lions,7.3831,52.0874,44.7043
2,Carlton,2.4978,55.7985,53.3007
16,West Coast,2.4038,42.1375,39.7337
13,Richmond,1.0612,44.7522,43.6910
8,Greater Western Sydney,-0.5561,49.0147,49.5708
7,Gold Coast,-1.0501,47.8524,48.9025


In [24]:
round_id = "202109"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 21.11
Overall MAE: 27.28


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,16.4805,58.3124,41.8319
10,Melbourne,13.7421,51.6061,37.8640
6,Geelong,12.7553,46.6258,33.8705
1,Brisbane Lions,12.6364,58.3472,45.7108
12,Port Adelaide,5.5700,50.8239,45.2539
16,West Coast,2.9795,50.5474,47.5680
8,Greater Western Sydney,0.5092,45.9953,45.4860
2,Carlton,0.1914,53.3547,53.1633
13,Richmond,-0.0546,37.9104,37.9650
5,Fremantle,-0.7056,47.7711,48.4766


In [25]:
round_id = "202110"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 23.94
Overall MAE: 26.94


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,24.0119,66.0815,42.0696
1,Brisbane Lions,14.2196,57.8978,43.6782
10,Melbourne,13.6116,55.0156,41.4040
6,Geelong,12.2539,51.6183,39.3644
12,Port Adelaide,4.6066,47.6757,43.0691
8,Greater Western Sydney,2.7750,48.8826,46.1075
5,Fremantle,1.8113,52.6161,50.8047
2,Carlton,0.5825,57.0782,56.4957
16,West Coast,0.3648,51.3866,51.0218
4,Essendon,-0.2816,54.2437,54.5252


In [26]:
round_id = "202111"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 27.00
Overall MAE: 26.95


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,19.3930,63.4969,44.1040
1,Brisbane Lions,17.0626,63.4435,46.3809
10,Melbourne,14.4682,62.1123,47.6441
6,Geelong,12.0505,48.2336,36.1832
12,Port Adelaide,5.8511,53.1675,47.3164
4,Essendon,5.1291,58.7593,53.6302
5,Fremantle,-0.0749,54.5210,54.5959
2,Carlton,-0.3309,59.6713,60.0022
13,Richmond,-1.6432,46.0966,47.7398
8,Greater Western Sydney,-1.7674,48.2495,50.0169


In [27]:
round_id = "202112"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 21.02
Overall MAE: 26.45


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,22.1707,68.0337,45.8630
10,Melbourne,15.8317,65.0566,49.2249
1,Brisbane Lions,12.5460,71.2108,58.6648
6,Geelong,12.0505,48.2336,36.1832
12,Port Adelaide,5.8511,53.1675,47.3164
4,Essendon,3.6616,61.8283,58.1667
2,Carlton,0.2462,59.6582,59.4120
13,Richmond,-0.5243,55.8317,56.3560
15,Sydney,-1.3313,61.5530,62.8843
8,Greater Western Sydney,-1.7674,48.2495,50.0169


In [28]:
round_id = "202113"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 22.21
Overall MAE: 26.13


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,22.1707,68.0337,45.8630
10,Melbourne,13.4527,66.4328,52.9802
1,Brisbane Lions,12.5460,71.2108,58.6648
6,Geelong,11.5621,57.9758,46.4136
12,Port Adelaide,4.5493,55.0667,50.5174
4,Essendon,3.6616,61.8283,58.1667
13,Richmond,1.0200,58.2522,57.2322
2,Carlton,0.2462,59.6582,59.4120
8,Greater Western Sydney,-0.5703,55.7333,56.3036
7,Gold Coast,-2.8604,55.7948,58.6552


In [29]:
round_id = "202114"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 20.12
Overall MAE: 25.70


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,18.2534,67.7048,49.4514
1,Brisbane Lions,15.2071,65.8231,50.6160
10,Melbourne,13.4527,66.4328,52.9802
6,Geelong,12.1061,61.6746,49.5685
12,Port Adelaide,8.5072,58.5379,50.0306
4,Essendon,3.1456,59.1271,55.9815
13,Richmond,1.0200,58.2522,57.2322
2,Carlton,0.2210,61.2821,61.0611
8,Greater Western Sydney,-0.5128,58.0877,58.6005
5,Fremantle,-3.8363,44.0098,47.8460


In [30]:
round_id = "202115"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 21.68
Overall MAE: 25.43


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,23.3044,69.9393,46.6349
1,Brisbane Lions,17.4648,75.5380,58.0732
10,Melbourne,13.7490,70.9901,57.2411
12,Port Adelaide,8.9185,55.9934,47.0749
6,Geelong,7.1171,61.5280,54.4110
2,Carlton,2.2768,67.2287,64.9519
4,Essendon,1.1894,63.9789,62.7894
13,Richmond,-0.7748,57.6842,58.4590
8,Greater Western Sydney,-2.5226,58.5656,61.0882
5,Fremantle,-3.6804,53.9355,57.6159


In [31]:
round_id = "202116"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 26.32
Overall MAE: 25.49


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,22.5323,73.2488,50.7165
1,Brisbane Lions,17.5353,74.6820,57.1467
10,Melbourne,12.4740,71.9886,59.5146
12,Port Adelaide,10.7115,59.1355,48.4240
6,Geelong,9.2131,62.9133,53.7001
2,Carlton,1.2490,64.0273,62.7783
15,Sydney,1.0451,66.6266,65.5814
4,Essendon,-1.7372,61.9628,63.7001
8,Greater Western Sydney,-2.3702,64.5204,66.8905
5,Fremantle,-2.5122,61.2425,63.7547


In [32]:
round_id = "202117"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 38.40
Overall MAE: 26.25


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,19.7591,68.1457,48.3867
10,Melbourne,15.2282,77.6773,62.4491
1,Brisbane Lions,14.4241,80.1063,65.6822
6,Geelong,8.3359,59.9796,51.6437
12,Port Adelaide,5.6387,64.3865,58.7478
4,Essendon,4.6719,65.3320,60.6601
5,Fremantle,2.4326,63.5361,61.1035
15,Sydney,1.4606,67.0814,65.6208
2,Carlton,1.0800,67.5015,66.4215
3,Collingwood,-0.2394,59.1785,59.4180


In [33]:
round_id = "202118"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 31.48
Overall MAE: 26.54


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,15.9213,71.2185,55.2972
10,Melbourne,15.5642,77.9295,62.3653
6,Geelong,13.6824,64.7844,51.1021
1,Brisbane Lions,12.8491,78.1466,65.2975
12,Port Adelaide,6.8454,61.2837,54.4383
4,Essendon,5.3433,66.7260,61.3827
2,Carlton,4.9229,70.2441,65.3211
15,Sydney,1.4620,63.1295,61.6674
8,Greater Western Sydney,-1.8090,60.5636,62.3726
7,Gold Coast,-2.2962,57.9629,60.2591


In [34]:
round_id = "202119"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 27.17
Overall MAE: 26.57


,Team,Rating,Offensive_Value,Defensive_Value
1,Brisbane Lions,16.5218,82.7084,66.1866
17,Western Bulldogs,14.3193,73.1901,58.8709
10,Melbourne,14.0177,78.4645,64.4468
6,Geelong,13.7270,70.8833,57.1563
12,Port Adelaide,8.6473,68.5704,59.9231
15,Sydney,4.4137,71.1390,66.7254
2,Carlton,3.8312,71.2967,67.4655
4,Essendon,3.8217,68.3214,64.4997
8,Greater Western Sydney,-0.6409,60.9009,61.5418
14,St Kilda,-4.6059,57.8703,62.4762


In [35]:
round_id = "202120"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 32.42
Overall MAE: 26.86


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,20.7323,80.6683,59.9360
17,Western Bulldogs,16.4973,75.0285,58.5312
6,Geelong,16.1083,65.7925,49.6842
1,Brisbane Lions,15.2710,83.8881,68.6171
12,Port Adelaide,9.3690,74.0760,64.7071
15,Sydney,4.1122,76.6144,72.5023
4,Essendon,3.2997,69.3210,66.0213
2,Carlton,2.5908,64.9474,62.3567
8,Greater Western Sydney,-2.1632,66.0265,68.1897
3,Collingwood,-2.7225,60.7676,63.4902


In [36]:
round_id = "202121"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 31.92
Overall MAE: 27.10


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,21.2086,76.9651,55.7565
1,Brisbane Lions,19.5808,80.0961,60.5153
17,Western Bulldogs,18.5499,80.1643,61.6144
6,Geelong,14.8759,68.1349,53.2590
12,Port Adelaide,8.7809,77.9770,69.1961
2,Carlton,1.0249,66.4171,65.3922
15,Sydney,0.8275,77.2130,76.3855
14,St Kilda,-0.0857,65.8873,65.9731
4,Essendon,-0.7326,67.9200,68.6525
13,Richmond,-2.1170,57.5797,59.6967


In [37]:
round_id = "202122"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 29.69
Overall MAE: 27.22


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,22.0449,74.5634,52.5184
1,Brisbane Lions,22.0106,87.8728,65.8623
6,Geelong,15.3136,70.0637,54.7501
12,Port Adelaide,14.2913,83.3891,69.0978
17,Western Bulldogs,14.2447,82.0319,67.7872
4,Essendon,4.6108,74.6956,70.0847
15,Sydney,2.2296,68.2814,66.0518
8,Greater Western Sydney,0.2103,62.7250,62.5148
14,St Kilda,-2.0024,64.9389,66.9413
13,Richmond,-4.2083,57.5491,61.7574


In [38]:
round_id = "202123"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 32.62
Overall MAE: 27.46


,Team,Rating,Offensive_Value,Defensive_Value
1,Brisbane Lions,21.9020,88.8456,66.9437
10,Melbourne,19.8148,81.0300,61.2152
12,Port Adelaide,14.7561,81.6270,66.8709
6,Geelong,13.8078,73.0132,59.2054
17,Western Bulldogs,10.9263,78.6308,67.7044
15,Sydney,7.0164,74.6431,67.6267
4,Essendon,6.9632,77.7885,70.8253
8,Greater Western Sydney,3.7118,72.9718,69.2601
14,St Kilda,0.3391,61.5406,61.2014
13,Richmond,-2.1427,57.4322,59.5749


In [39]:
round_id = "2021F1"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 17.36
Overall MAE: 27.04


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,20.3645,83.7015,63.3370
1,Brisbane Lions,17.1806,92.8847,75.7041
12,Port Adelaide,16.4972,84.0672,67.5699
17,Western Bulldogs,11.4696,78.7996,67.3300
6,Geelong,9.2103,74.8314,65.6211
15,Sydney,9.2062,74.7299,65.5237
4,Essendon,4.6310,74.3096,69.6786
8,Greater Western Sydney,0.4492,74.2555,73.8063
14,St Kilda,0.3391,61.5406,61.2014
13,Richmond,-2.1427,57.4322,59.5749


In [40]:
round_id = "2021F2"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 20.50
Overall MAE: 26.77


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,20.3645,83.7015,63.3370
12,Port Adelaide,16.4972,84.0672,67.5699
1,Brisbane Lions,16.3196,92.2476,75.9280
6,Geelong,11.3216,80.0266,68.7051
17,Western Bulldogs,9.4656,79.3993,69.9337
15,Sydney,9.2062,74.7299,65.5237
4,Essendon,4.6310,74.3096,69.6786
14,St Kilda,0.3391,61.5406,61.2014
13,Richmond,-2.1427,57.4322,59.5749
8,Greater Western Sydney,-2.6280,64.7368,67.3648


In [41]:
round_id = "2021F3"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 70.09
Overall MAE: 28.44


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,24.9006,90.7675,65.8669
1,Brisbane Lions,16.3196,92.2476,75.9280
17,Western Bulldogs,12.2823,78.5426,66.2603
12,Port Adelaide,11.0842,81.4994,70.4151
15,Sydney,9.2062,74.7299,65.5237
4,Essendon,4.6310,74.3096,69.6786
6,Geelong,3.6168,77.5791,73.9623
14,St Kilda,0.3391,61.5406,61.2014
13,Richmond,-2.1427,57.4322,59.5749
8,Greater Western Sydney,-2.6280,64.7368,67.3648


In [42]:
round_id = "2021F4"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 37.85
Overall MAE: 28.79


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,27.4316,91.6556,64.2241
1,Brisbane Lions,16.3196,92.2476,75.9280
12,Port Adelaide,11.0842,81.4994,70.4151
15,Sydney,9.2062,74.7299,65.5237
17,Western Bulldogs,6.0331,80.0372,74.0042
4,Essendon,4.6310,74.3096,69.6786
6,Geelong,3.6168,77.5791,73.9623
14,St Kilda,0.3391,61.5406,61.2014
13,Richmond,-2.1427,57.4322,59.5749
8,Greater Western Sydney,-2.6280,64.7368,67.3648


In [43]:
round_id = "202201"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 23.13
Overall MAE: 28.59


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,29.2090,86.1902,56.9812
1,Brisbane Lions,16.3537,87.1412,70.7876
15,Sydney,8.4839,72.7691,64.2851
12,Port Adelaide,8.3098,75.1902,66.8804
6,Geelong,7.2606,74.7294,67.4688
17,Western Bulldogs,0.9092,77.5532,76.6440
4,Essendon,0.1624,69.6692,69.5068
14,St Kilda,0.1266,58.9509,58.8242
8,Greater Western Sydney,-2.5636,71.2371,73.8008
13,Richmond,-4.2293,70.4099,74.6392


In [44]:
round_id = "202202"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 28.50
Overall MAE: 28.58


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,25.7579,83.8417,58.0838
1,Brisbane Lions,14.3885,84.8738,70.4853
15,Sydney,7.2823,73.7886,66.5063
6,Geelong,6.8878,78.3701,71.4823
12,Port Adelaide,6.1526,65.9698,59.8172
17,Western Bulldogs,2.2193,76.3165,74.0973
14,St Kilda,1.8445,57.6144,55.7698
4,Essendon,0.4760,79.5252,79.0493
13,Richmond,-1.6916,61.1154,62.8070
3,Collingwood,-3.5810,67.2723,70.8533


In [45]:
round_id = "202203"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 25.83
Overall MAE: 28.49


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,26.9377,84.0207,57.0830
1,Brisbane Lions,21.6719,92.1730,70.5011
12,Port Adelaide,6.5460,67.0935,60.5475
17,Western Bulldogs,5.6985,79.7402,74.0418
14,St Kilda,5.5727,64.8171,59.2445
6,Geelong,5.0464,79.0185,73.9721
15,Sydney,2.8529,71.6026,68.7497
8,Greater Western Sydney,-2.0495,69.8821,71.9316
3,Collingwood,-2.0703,72.6864,74.7567
4,Essendon,-3.3273,75.1998,78.5271


In [46]:
round_id = "202204"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 25.85
Overall MAE: 28.41


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,27.0658,87.1463,60.0805
1,Brisbane Lions,17.0824,86.8900,69.8075
14,St Kilda,8.4169,68.0928,59.6760
6,Geelong,6.9640,73.1390,66.1750
17,Western Bulldogs,4.3875,75.3477,70.9602
12,Port Adelaide,3.0696,67.6419,64.5723
15,Sydney,2.6314,75.1524,72.5210
3,Collingwood,1.5471,65.5337,63.9865
5,Fremantle,-2.1920,63.5851,65.7772
4,Essendon,-3.9149,68.3396,72.2545


In [47]:
round_id = "202205"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 29.29
Overall MAE: 28.43


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,28.0833,90.6202,62.5369
1,Brisbane Lions,16.7858,91.0839,74.2980
17,Western Bulldogs,9.6211,84.3614,74.7404
14,St Kilda,8.9318,65.8269,56.8952
15,Sydney,7.6360,72.0449,64.4089
6,Geelong,6.8668,85.1203,78.2535
12,Port Adelaide,1.7722,67.9088,66.1366
5,Fremantle,1.1204,66.6714,65.5510
3,Collingwood,-0.0192,67.9557,67.9749
13,Richmond,-2.4559,76.8133,79.2691


In [48]:
round_id = "202206"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 30.31
Overall MAE: 28.49


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,30.7999,88.4653,57.6654
1,Brisbane Lions,16.2164,88.6477,72.4313
6,Geelong,11.6462,80.8527,69.2064
15,Sydney,11.1514,79.0706,67.9192
14,St Kilda,9.3445,68.9579,59.6134
12,Port Adelaide,6.5031,77.4962,70.9930
17,Western Bulldogs,5.3962,74.7969,69.4007
5,Fremantle,3.8875,70.9433,67.0558
3,Collingwood,-1.2341,73.4559,74.6900
0,Adelaide,-4.6181,66.7245,71.3426


In [49]:
round_id = "202207"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 33.17
Overall MAE: 28.63


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,28.3705,87.5124,59.1419
1,Brisbane Lions,15.5947,90.4765,74.8817
6,Geelong,9.3987,80.3984,70.9997
14,St Kilda,9.2296,70.9581,61.7285
15,Sydney,9.0363,84.4566,75.4203
17,Western Bulldogs,7.1399,76.6470,69.5071
12,Port Adelaide,5.0333,77.8560,72.8227
5,Fremantle,4.5817,67.5849,63.0032
13,Richmond,3.3685,70.7464,67.3779
3,Collingwood,1.9441,73.7479,71.8038


In [50]:
round_id = "202208"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 28.46
Overall MAE: 28.62


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,27.9109,93.0358,65.1249
1,Brisbane Lions,18.8291,84.0963,65.2672
6,Geelong,12.0505,70.2999,58.2494
5,Fremantle,10.2996,62.2369,51.9372
12,Port Adelaide,7.1518,80.4579,73.3061
15,Sydney,6.3173,78.5861,72.2688
14,St Kilda,5.9292,75.4838,69.5546
13,Richmond,4.0097,78.0888,74.0791
2,Carlton,3.9648,81.3386,77.3738
17,Western Bulldogs,3.8041,72.9201,69.1160


In [51]:
round_id = "202209"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 31.20
Overall MAE: 28.69


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,30.7228,94.3609,63.6382
1,Brisbane Lions,17.6942,80.9703,63.2761
12,Port Adelaide,11.9633,80.1377,68.1744
15,Sydney,10.2956,82.6464,72.3508
5,Fremantle,9.1583,67.7215,58.5632
14,St Kilda,8.1351,80.6837,72.5485
6,Geelong,8.0466,81.0631,73.0165
17,Western Bulldogs,7.7697,78.4558,70.6861
2,Carlton,7.3040,80.7667,73.4627
13,Richmond,7.0886,85.5039,78.4153


In [52]:
round_id = "202210"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 17.84
Overall MAE: 28.40


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,31.8756,92.4521,60.5765
1,Brisbane Lions,15.3243,82.4413,67.1170
13,Richmond,11.5472,82.0602,70.5130
6,Geelong,10.2378,80.8664,70.6286
17,Western Bulldogs,10.2246,81.7582,71.5336
2,Carlton,8.6090,81.7743,73.1654
12,Port Adelaide,7.7711,78.9138,71.1428
15,Sydney,7.2307,80.9996,73.7689
5,Fremantle,5.2546,74.4337,69.1791
14,St Kilda,4.9092,70.4585,65.5493


In [53]:
round_id = "202211"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 32.19
Overall MAE: 28.50


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,26.4447,89.8490,63.4044
1,Brisbane Lions,17.4473,84.5997,67.1523
17,Western Bulldogs,17.3400,89.3930,72.0530
6,Geelong,11.1073,72.2615,61.1542
13,Richmond,9.1460,72.6305,63.4845
15,Sydney,7.7541,77.9389,70.1848
12,Port Adelaide,7.5917,83.9942,76.4026
14,St Kilda,7.5148,68.2081,60.6933
5,Fremantle,6.9725,76.5940,69.6214
2,Carlton,4.7366,81.2655,76.5288


In [54]:
round_id = "202212"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 30.55
Overall MAE: 28.55


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,22.4932,85.4694,62.9762
1,Brisbane Lions,14.9444,86.8065,71.8621
17,Western Bulldogs,14.5743,92.7318,78.1575
6,Geelong,11.0283,79.7175,68.6892
13,Richmond,9.1460,72.6305,63.4845
15,Sydney,8.2858,79.4463,71.1606
12,Port Adelaide,7.5917,83.9942,76.4026
14,St Kilda,7.5148,68.2081,60.6933
5,Fremantle,7.0335,75.0919,68.0584
7,Gold Coast,6.0651,79.0410,72.9759


In [55]:
round_id = "202213"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 8.91
Overall MAE: 28.06


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,16.8726,82.4622,65.5895
1,Brisbane Lions,15.8799,91.3669,75.4870
17,Western Bulldogs,14.5743,92.7318,78.1575
13,Richmond,11.9887,82.8603,70.8716
6,Geelong,11.0283,79.7175,68.6892
15,Sydney,8.2858,79.4463,71.1606
5,Fremantle,7.4015,84.5941,77.1926
3,Collingwood,6.8952,86.0831,79.1880
7,Gold Coast,6.0651,79.0410,72.9759
2,Carlton,5.3802,84.0821,78.7020


In [56]:
round_id = "202214"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 19.86
Overall MAE: 27.86


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,16.8726,82.4622,65.5895
17,Western Bulldogs,16.6016,93.3689,76.7673
1,Brisbane Lions,15.8799,91.3669,75.4870
6,Geelong,12.5855,80.4537,67.8682
13,Richmond,11.4206,86.3915,74.9709
7,Gold Coast,8.0925,84.4777,76.3852
5,Fremantle,7.4015,84.5941,77.1926
3,Collingwood,6.8952,86.0831,79.1880
15,Sydney,5.9301,83.3488,77.4187
12,Port Adelaide,4.2948,75.7332,71.4384


In [57]:
round_id = "202215"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 23.01
Overall MAE: 27.75


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,20.2352,88.0388,67.8036
17,Western Bulldogs,18.2572,88.9101,70.6529
6,Geelong,11.5835,87.5600,75.9764
13,Richmond,10.0219,84.3514,74.3294
3,Collingwood,9.6561,81.5592,71.9031
1,Brisbane Lions,9.2420,78.7905,69.5485
15,Sydney,9.1120,77.3283,68.2163
7,Gold Coast,7.5851,85.3320,77.7469
5,Fremantle,5.3840,81.8189,76.4349
2,Carlton,5.0677,81.3279,76.2603


In [58]:
round_id = "202216"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 24.76
Overall MAE: 27.68


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,22.2417,83.0583,60.8166
6,Geelong,20.0744,85.7290,65.6546
17,Western Bulldogs,14.5826,81.2531,66.6705
1,Brisbane Lions,10.1667,89.3286,79.1618
15,Sydney,9.8867,83.2994,73.4127
3,Collingwood,9.8355,85.0614,75.2260
13,Richmond,9.5891,83.4761,73.8870
2,Carlton,6.3824,84.9737,78.5913
5,Fremantle,6.2459,85.0293,78.7834
7,Gold Coast,5.6816,77.2839,71.6022


In [59]:
round_id = "202217"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 26.63
Overall MAE: 27.65


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,22.7676,91.0897,68.3221
10,Melbourne,15.3169,89.0539,73.7370
2,Carlton,11.5947,89.3811,77.7864
15,Sydney,11.3222,82.6471,71.3250
13,Richmond,11.1659,82.5546,71.3886
17,Western Bulldogs,10.7002,88.2056,77.5054
3,Collingwood,10.2684,79.2194,68.9510
5,Fremantle,8.0518,88.4826,80.4307
1,Brisbane Lions,7.5492,68.9340,61.3848
12,Port Adelaide,3.9693,81.1963,77.2271


In [60]:
round_id = "202218"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 26.87
Overall MAE: 27.64


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,23.6163,90.6415,67.0252
10,Melbourne,13.8255,82.3705,68.5450
13,Richmond,12.5291,80.4151,67.8860
15,Sydney,11.7922,83.6594,71.8671
1,Brisbane Lions,10.5303,72.7362,62.2060
17,Western Bulldogs,9.5400,84.5279,74.9879
3,Collingwood,8.8612,76.0755,67.2143
2,Carlton,7.3097,84.3648,77.0551
5,Fremantle,5.6444,87.7440,82.0996
12,Port Adelaide,3.5320,83.2588,79.7268


In [61]:
round_id = "202219"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 13.50
Overall MAE: 27.33


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,21.1247,85.7176,64.5930
15,Sydney,14.1459,86.7737,72.6277
10,Melbourne,12.8306,84.2226,71.3920
13,Richmond,11.8378,80.8455,69.0077
2,Carlton,10.3341,88.1761,77.8419
1,Brisbane Lions,9.8358,85.9306,76.0949
17,Western Bulldogs,8.1984,91.3124,83.1141
3,Collingwood,6.9675,76.5835,69.6160
5,Fremantle,4.5184,84.7158,80.1974
4,Essendon,3.7243,81.3178,77.5935


In [62]:
round_id = "202220"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 22.84
Overall MAE: 27.23


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,21.2729,89.0632,67.7902
15,Sydney,19.2669,89.8774,70.6105
10,Melbourne,15.0688,89.3005,74.2317
13,Richmond,10.4219,89.2592,78.8373
1,Brisbane Lions,9.0843,84.9584,75.8741
2,Carlton,7.1123,87.8163,80.7040
3,Collingwood,6.7888,78.5863,71.7975
17,Western Bulldogs,5.1178,88.7070,83.5892
4,Essendon,4.6084,73.7907,69.1823
12,Port Adelaide,2.4599,77.9184,75.4585


In [63]:
round_id = "202221"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 21.25
Overall MAE: 27.11


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,23.8531,87.8954,64.0423
15,Sydney,20.1846,91.1518,70.9672
10,Melbourne,15.0852,95.3188,80.2336
13,Richmond,13.4485,83.0322,69.5837
1,Brisbane Lions,10.2412,91.6090,81.3678
17,Western Bulldogs,5.1707,88.1483,82.9776
3,Collingwood,4.5866,78.3698,73.7832
2,Carlton,4.3358,81.9627,77.6269
4,Essendon,1.1029,81.5180,80.4151
5,Fremantle,-0.0739,78.2348,78.3087


In [64]:
round_id = "202222"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 30.24
Overall MAE: 27.17


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,27.1663,92.9868,65.8205
15,Sydney,21.3154,93.2538,71.9385
13,Richmond,15.4109,84.8617,69.4508
10,Melbourne,13.4195,93.7160,80.2965
1,Brisbane Lions,10.9844,92.3748,81.3904
17,Western Bulldogs,5.5143,87.2773,81.7629
2,Carlton,4.0594,77.2307,73.1712
5,Fremantle,3.9486,80.4244,76.4758
12,Port Adelaide,3.5505,80.2211,76.6706
3,Collingwood,0.9787,71.4275,70.4488


In [65]:
round_id = "202223"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 36.75
Overall MAE: 27.36


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,32.4269,89.7051,57.2782
15,Sydney,20.9996,91.7960,70.7964
13,Richmond,18.3674,89.3269,70.9594
10,Melbourne,14.2652,93.0955,78.8303
17,Western Bulldogs,9.4869,87.5607,78.0738
12,Port Adelaide,8.8342,85.1304,76.2962
1,Brisbane Lions,7.6983,88.6803,80.9820
5,Fremantle,6.6323,84.3070,77.6747
2,Carlton,5.3506,81.2376,75.8870
3,Collingwood,-0.8163,79.8179,80.6342


In [66]:
round_id = "2022F1"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 24.02
Overall MAE: 27.30


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,30.3482,94.2535,63.9053
15,Sydney,19.3665,92.0551,72.6886
13,Richmond,15.6599,92.8923,77.2324
10,Melbourne,12.3718,92.9039,80.5321
12,Port Adelaide,8.8342,85.1304,76.2962
17,Western Bulldogs,8.6433,81.5403,72.8970
1,Brisbane Lions,7.7992,87.3789,79.5798
5,Fremantle,5.8640,83.2197,77.3557
2,Carlton,5.3506,81.2376,75.8870
3,Collingwood,-1.8986,81.0432,82.9418


In [67]:
round_id = "2022F2"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 22.65
Overall MAE: 27.21


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,30.3482,94.2535,63.9053
15,Sydney,19.3665,92.0551,72.6886
13,Richmond,15.6599,92.8923,77.2324
10,Melbourne,12.8077,91.8819,79.0742
12,Port Adelaide,8.8342,85.1304,76.2962
17,Western Bulldogs,8.6433,81.5403,72.8970
2,Carlton,5.3506,81.2376,75.8870
1,Brisbane Lions,5.3462,70.5143,65.1681
5,Fremantle,3.0538,80.7550,77.7012
3,Collingwood,0.5150,77.9969,77.4819


In [68]:
round_id = "2022F3"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 29.13
Overall MAE: 27.25


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,30.4400,95.7400,65.3001
15,Sydney,18.7018,94.0454,75.3437
13,Richmond,15.6599,92.8923,77.2324
10,Melbourne,12.8077,91.8819,79.0742
12,Port Adelaide,8.8342,85.1304,76.2962
17,Western Bulldogs,8.6433,81.5403,72.8970
2,Carlton,5.3506,81.2376,75.8870
5,Fremantle,3.0538,80.7550,77.7012
1,Brisbane Lions,1.6850,78.0885,76.4035
3,Collingwood,-0.8084,79.7299,80.5384


In [69]:
round_id = "2022F4"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 38.16
Overall MAE: 27.45


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,33.1629,95.0267,61.8638
13,Richmond,15.6599,92.8923,77.2324
10,Melbourne,12.8077,91.8819,79.0742
15,Sydney,11.0647,88.1984,77.1337
12,Port Adelaide,8.8342,85.1304,76.2962
17,Western Bulldogs,8.6433,81.5403,72.8970
2,Carlton,5.3506,81.2376,75.8870
5,Fremantle,3.0538,80.7550,77.7012
1,Brisbane Lions,1.6850,78.0885,76.4035
3,Collingwood,-0.8084,79.7299,80.5384


In [70]:
round_id = "202301"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 32.99
Overall MAE: 27.55


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,27.7699,89.0031,61.2332
12,Port Adelaide,15.8851,80.6740,64.7889
13,Richmond,15.6603,95.5567,79.8964
10,Melbourne,15.3897,80.6816,65.2919
15,Sydney,14.3852,82.6965,68.3113
17,Western Bulldogs,3.9162,85.3083,81.3921
2,Carlton,3.2492,88.0430,84.7938
5,Fremantle,2.6264,81.3933,78.7669
3,Collingwood,1.3491,90.7094,89.3604
4,Essendon,-3.0846,81.7314,84.8160


In [71]:
round_id = "202302"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 31.51
Overall MAE: 27.62


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,23.2030,81.6993,58.4964
15,Sydney,17.3977,82.1686,64.7709
13,Richmond,14.1959,89.5633,75.3674
10,Melbourne,12.0877,84.3726,72.2850
12,Port Adelaide,10.2969,79.2621,68.9652
3,Collingwood,5.2139,88.4677,83.2538
2,Carlton,4.7142,85.0972,80.3831
5,Fremantle,3.4147,78.3121,74.8974
17,Western Bulldogs,0.8508,80.9475,80.0967
4,Essendon,-1.7620,79.0565,80.8185


In [72]:
round_id = "202303"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 19.93
Overall MAE: 27.48


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,19.6487,85.3449,65.6962
15,Sydney,14.1161,84.3695,70.2534
10,Melbourne,12.4208,82.7902,70.3694
13,Richmond,10.7247,74.6198,63.8951
12,Port Adelaide,7.3711,85.3534,77.9823
3,Collingwood,6.7441,85.3779,78.6337
2,Carlton,5.8910,86.4355,80.5445
5,Fremantle,4.4209,75.1312,70.7103
17,Western Bulldogs,1.0580,83.3111,82.2531
14,St Kilda,-1.6744,58.5999,60.2743


In [73]:
round_id = "202304"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 29.07
Overall MAE: 27.51


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,23.6680,90.2875,66.6194
10,Melbourne,15.2348,85.6102,70.3754
15,Sydney,13.6271,88.2126,74.5855
13,Richmond,8.3360,73.2742,64.9382
12,Port Adelaide,5.7114,76.4812,70.7699
2,Carlton,5.6504,79.9998,74.3495
3,Collingwood,4.1670,82.7524,78.5854
17,Western Bulldogs,2.2684,83.3062,81.0378
5,Fremantle,2.0813,72.3077,70.2264
14,St Kilda,1.3750,60.8247,59.4497


In [74]:
round_id = "202305"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 21.57
Overall MAE: 27.41


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,24.9348,93.5550,68.6201
15,Sydney,14.1797,81.2707,67.0909
10,Melbourne,10.8144,81.6975,70.8830
1,Brisbane Lions,7.2533,81.9870,74.7337
12,Port Adelaide,6.9918,77.9310,70.9392
13,Richmond,5.5870,72.4330,66.8460
3,Collingwood,4.1621,79.7384,75.5763
5,Fremantle,3.4274,78.9616,75.5342
4,Essendon,2.7077,93.2007,90.4929
0,Adelaide,1.3691,73.1912,71.8221


In [75]:
round_id = "202306"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 28.24
Overall MAE: 27.42


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,28.5738,92.0442,63.4704
12,Port Adelaide,9.7797,72.1132,62.3335
10,Melbourne,9.3372,83.8379,74.5007
1,Brisbane Lions,9.1132,86.1711,77.0579
15,Sydney,6.6293,78.4929,71.8636
13,Richmond,5.4241,78.8161,73.3920
17,Western Bulldogs,5.1829,87.7643,82.5814
3,Collingwood,4.9859,79.0836,74.0977
4,Essendon,1.1970,82.3415,81.1445
14,St Kilda,1.1673,71.5559,70.3885


In [76]:
round_id = "202307"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 31.32
Overall MAE: 27.49


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,28.5222,90.2769,61.7547
10,Melbourne,13.3255,87.0125,73.6870
1,Brisbane Lions,12.0754,81.7964,69.7211
2,Carlton,8.9127,91.0750,82.1623
12,Port Adelaide,8.8543,71.7713,62.9170
15,Sydney,7.5162,80.2442,72.7280
17,Western Bulldogs,6.6021,84.6947,78.0926
13,Richmond,3.2613,73.9998,70.7385
3,Collingwood,2.9054,75.8053,72.8999
0,Adelaide,2.5436,71.9491,69.4054


In [77]:
round_id = "202308"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 20.40
Overall MAE: 27.37


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,27.7674,86.2195,58.4521
1,Brisbane Lions,12.4930,77.1778,64.6848
12,Port Adelaide,12.1450,78.4470,66.3020
10,Melbourne,11.9123,89.3251,77.4127
2,Carlton,6.3962,85.4963,79.1001
13,Richmond,6.0670,85.4038,79.3367
15,Sydney,6.0012,83.7553,77.7541
17,Western Bulldogs,5.9046,88.9882,83.0836
3,Collingwood,3.3781,83.6485,80.2704
14,St Kilda,2.3235,72.6848,70.3613


In [78]:
round_id = "202309"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 34.07
Overall MAE: 27.48


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,25.3023,77.6322,52.3299
12,Port Adelaide,15.6511,73.0933,57.4422
1,Brisbane Lions,15.0612,79.4118,64.3506
10,Melbourne,14.8459,89.9684,75.1225
7,Gold Coast,6.4034,72.0618,65.6583
3,Collingwood,5.6872,81.0405,75.3533
17,Western Bulldogs,5.6823,84.3760,78.6937
2,Carlton,5.3884,87.2090,81.8207
13,Richmond,5.1487,88.1414,82.9927
5,Fremantle,3.2092,80.6407,77.4315


In [79]:
round_id = "202310"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 31.86
Overall MAE: 27.55


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,21.4665,76.1515,54.6850
1,Brisbane Lions,17.5054,83.2637,65.7583
12,Port Adelaide,16.2891,76.4396,60.1505
10,Melbourne,11.1581,84.9679,73.8098
17,Western Bulldogs,9.7163,85.3029,75.5866
3,Collingwood,7.1635,86.2368,79.0733
13,Richmond,4.7823,85.0549,80.2726
5,Fremantle,4.1939,82.4544,78.2605
15,Sydney,3.3294,76.0946,72.7652
2,Carlton,2.8045,85.5999,82.7954


In [80]:
round_id = "202311"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 18.43
Overall MAE: 27.41


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,18.0660,78.0066,59.9406
12,Port Adelaide,17.5068,75.0008,57.4941
1,Brisbane Lions,15.6376,82.7822,67.1445
10,Melbourne,11.6973,79.5557,67.8583
17,Western Bulldogs,9.6380,87.7862,78.1482
3,Collingwood,7.8752,86.0888,78.2135
15,Sydney,2.8752,79.7072,76.8320
2,Carlton,2.6453,81.2551,78.6098
5,Fremantle,2.1195,78.7995,76.6800
13,Richmond,1.4575,80.6473,79.1898


In [81]:
round_id = "202312"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 16.90
Overall MAE: 27.25


,Team,Rating,Offensive_Value,Defensive_Value
12,Port Adelaide,18.7896,82.3947,63.6051
1,Brisbane Lions,15.6376,82.7822,67.1445
6,Geelong,15.3658,76.8247,61.4588
10,Melbourne,14.1324,77.9685,63.8361
3,Collingwood,12.2508,85.7707,73.5199
17,Western Bulldogs,9.5678,89.5747,80.0069
15,Sydney,2.8752,79.7072,76.8320
13,Richmond,2.7374,78.8337,76.0963
5,Fremantle,2.1195,78.7995,76.6800
7,Gold Coast,2.0181,79.4038,77.3857


In [82]:
round_id = "202313"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 31.28
Overall MAE: 27.31


,Team,Rating,Offensive_Value,Defensive_Value
12,Port Adelaide,19.3923,90.0445,70.6522
10,Melbourne,15.3751,80.1286,64.7536
6,Geelong,15.3658,76.8247,61.4588
1,Brisbane Lions,13.3568,79.6867,66.3299
3,Collingwood,8.3698,77.1697,68.7998
17,Western Bulldogs,6.1293,85.7378,79.6085
0,Adelaide,5.9164,72.4899,66.5735
13,Richmond,2.6456,83.8701,81.2245
14,St Kilda,2.3443,69.1229,66.7786
7,Gold Coast,2.0181,79.4038,77.3857


In [83]:
round_id = "202314"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 33.84
Overall MAE: 27.41


,Team,Rating,Offensive_Value,Defensive_Value
12,Port Adelaide,21.3373,92.0575,70.7203
1,Brisbane Lions,15.9965,69.7027,53.7062
10,Melbourne,15.3751,80.1286,64.7536
6,Geelong,9.9450,91.0071,81.0621
3,Collingwood,8.3698,77.1697,68.7998
17,Western Bulldogs,8.2931,91.6195,83.3264
0,Adelaide,5.9164,72.4899,66.5735
2,Carlton,4.0392,80.5940,76.5548
13,Richmond,3.6144,81.1110,77.4966
14,St Kilda,0.8765,67.1223,66.2458


In [84]:
round_id = "202315"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 51.15
Overall MAE: 27.75


,Team,Rating,Offensive_Value,Defensive_Value
12,Port Adelaide,21.3373,92.0575,70.7203
1,Brisbane Lions,15.8901,71.2626,55.3725
10,Melbourne,13.6100,80.2687,66.6588
15,Sydney,9.6248,89.9421,80.3174
6,Geelong,9.1782,85.3602,76.1820
17,Western Bulldogs,8.2931,91.6195,83.3264
3,Collingwood,7.9745,81.1762,73.2017
0,Adelaide,4.8831,74.7896,69.9065
2,Carlton,4.0392,80.5940,76.5548
13,Richmond,3.6144,81.1110,77.4966


In [85]:
round_id = "202316"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 39.03
Overall MAE: 27.91


,Team,Rating,Offensive_Value,Defensive_Value
1,Brisbane Lions,20.5870,77.0615,56.4745
12,Port Adelaide,18.2042,91.3645,73.1603
10,Melbourne,15.6033,83.7179,68.1146
3,Collingwood,13.3740,87.0718,73.6978
15,Sydney,12.2256,84.3856,72.1601
0,Adelaide,10.1658,74.7124,64.5465
2,Carlton,8.1763,86.0411,77.8648
17,Western Bulldogs,7.4949,86.7006,79.2057
6,Geelong,4.6971,87.8733,83.1762
14,St Kilda,3.7452,68.1137,64.3684


In [86]:
round_id = "202317"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 30.72
Overall MAE: 27.95


,Team,Rating,Offensive_Value,Defensive_Value
1,Brisbane Lions,26.5300,77.8548,51.3248
12,Port Adelaide,20.0349,89.4592,69.4243
10,Melbourne,13.1935,75.1318,61.9384
3,Collingwood,12.1396,88.5417,76.4021
6,Geelong,10.9614,85.1732,74.2118
2,Carlton,10.5249,86.5611,76.0362
15,Sydney,7.5838,83.7078,76.1240
17,Western Bulldogs,6.6424,87.7841,81.1418
0,Adelaide,6.5038,76.3866,69.8828
14,St Kilda,4.2203,62.2583,58.0380


In [87]:
round_id = "202318"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 24.44
Overall MAE: 27.90


,Team,Rating,Offensive_Value,Defensive_Value
1,Brisbane Lions,24.9653,81.9435,56.9783
6,Geelong,16.5807,89.5404,72.9597
12,Port Adelaide,15.8730,85.7708,69.8978
3,Collingwood,12.9799,89.2532,76.2733
2,Carlton,11.6308,85.7659,74.1350
10,Melbourne,10.7858,71.6468,60.8610
15,Sydney,7.5053,80.3035,72.7982
17,Western Bulldogs,5.2982,82.6013,77.3031
0,Adelaide,4.9911,78.5811,73.5900
13,Richmond,3.7337,82.0582,78.3245


In [88]:
round_id = "202319"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 20.17
Overall MAE: 27.80


,Team,Rating,Offensive_Value,Defensive_Value
1,Brisbane Lions,23.0034,74.2295,51.2261
12,Port Adelaide,17.0443,91.0532,74.0090
2,Carlton,16.8536,69.3488,52.4952
6,Geelong,14.3880,85.9229,71.5349
10,Melbourne,10.3304,71.9468,61.6164
3,Collingwood,8.9234,92.4896,83.5662
15,Sydney,8.5702,90.5860,82.0158
17,Western Bulldogs,7.1123,91.8349,84.7226
13,Richmond,6.1687,81.3203,75.1517
8,Greater Western Sydney,3.9896,73.8535,69.8639


In [89]:
round_id = "202320"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 28.61
Overall MAE: 27.81


,Team,Rating,Offensive_Value,Defensive_Value
1,Brisbane Lions,18.5792,73.1359,54.5567
2,Carlton,14.3904,75.6794,61.2890
10,Melbourne,12.8580,74.6597,61.8017
6,Geelong,12.2484,89.8278,77.5794
12,Port Adelaide,12.1897,89.8883,77.6986
3,Collingwood,8.8089,91.5307,82.7218
15,Sydney,8.1380,92.7864,84.6484
0,Adelaide,6.6321,72.4920,65.8599
17,Western Bulldogs,5.8972,86.3811,80.4840
14,St Kilda,4.9112,62.8704,57.9591


In [90]:
round_id = "202321"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 22.97
Overall MAE: 27.74


,Team,Rating,Offensive_Value,Defensive_Value
1,Brisbane Lions,17.1301,74.4099,57.2797
2,Carlton,14.4705,70.5038,56.0333
10,Melbourne,13.1316,80.0098,66.8782
6,Geelong,11.6628,87.9515,76.2887
17,Western Bulldogs,10.5474,88.5525,78.0051
12,Port Adelaide,10.3315,66.9429,56.6114
0,Adelaide,10.3071,69.1388,58.8317
15,Sydney,6.2726,91.0436,84.7710
8,Greater Western Sydney,4.7367,84.6088,79.8721
3,Collingwood,4.5628,86.2269,81.6642


In [91]:
round_id = "202322"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 25.33
Overall MAE: 27.71


,Team,Rating,Offensive_Value,Defensive_Value
1,Brisbane Lions,14.3829,78.3365,63.9536
2,Carlton,12.6624,71.3044,58.6420
12,Port Adelaide,12.1879,82.8976,70.7096
10,Melbourne,12.1794,84.2254,72.0459
0,Adelaide,10.3105,68.1386,57.8280
6,Geelong,8.7559,83.4557,74.6997
17,Western Bulldogs,8.4573,88.4884,80.0311
15,Sydney,7.5448,86.6767,79.1319
3,Collingwood,5.8471,93.1069,87.2598
14,St Kilda,3.9292,70.7085,66.7793


In [92]:
round_id = "202323"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 30.30
Overall MAE: 27.74


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,14.2211,80.4918,66.2707
1,Brisbane Lions,13.3991,83.7386,70.3395
12,Port Adelaide,12.2723,81.5722,69.2999
2,Carlton,11.9968,78.6517,66.6550
8,Greater Western Sydney,9.4932,85.5129,76.0197
0,Adelaide,8.8210,70.9359,62.1149
14,St Kilda,8.4066,72.8856,64.4790
17,Western Bulldogs,7.6767,84.7130,77.0363
15,Sydney,7.2487,88.1849,80.9362
3,Collingwood,4.8079,84.9231,80.1152


In [93]:
round_id = "202324"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 26.19
Overall MAE: 27.72


,Team,Rating,Offensive_Value,Defensive_Value
1,Brisbane Lions,16.6130,79.9360,63.3230
0,Adelaide,14.5966,68.4918,53.8952
12,Port Adelaide,14.2237,79.2363,65.0126
10,Melbourne,13.2831,78.9685,65.6854
8,Greater Western Sydney,11.0151,85.1777,74.1626
2,Carlton,8.3258,80.0624,71.7366
3,Collingwood,7.1800,89.9567,82.7767
15,Sydney,6.0397,86.3646,80.3249
17,Western Bulldogs,5.6859,81.8324,76.1464
6,Geelong,3.9321,61.6066,57.6745


In [86]:
round_id = "2023F1"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: nan
Overall MAE: nan


,Team,Rating,Offensive_Value,Defensive_Value
2,Carlton,12.5949,97.8698,85.2749
10,Melbourne,12.5291,110.2879,97.7588
12,Port Adelaide,12.1954,108.4921,96.2966
1,Brisbane Lions,11.3704,102.5713,91.2009
0,Adelaide,10.3000,93.2300,82.9300
14,St Kilda,9.5547,94.7783,85.2237
8,Greater Western Sydney,9.2834,111.0708,101.7874
17,Western Bulldogs,8.7621,112.1202,103.3580
15,Sydney,7.4716,118.5591,111.0874
3,Collingwood,5.2744,104.9925,99.7181


In [ ]:
round_id = "2023F2"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

In [ ]:
round_id = "2023F3"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

In [ ]:
round_id = "2023F4"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)